In [1]:
import requests
import cssutils
from bs4 import BeautifulSoup
import os
import json
import pandas as pd

link = 'https://store.steampowered.com/search/results'
game = input('Enter game you want search: ')
head = {'cookie': 'sessionid=5ae342dee8ccf71fc64e6471'}

In [2]:
def get_pagination(game):
    param = {
        'term': game,
        'page': 1,
    }
    total_item = 0
    req = requests.get(link, headers=head, params=param)
    soup = BeautifulSoup(req.text, 'html.parser')
    page_item = soup.find('div', 'search_pagination_right').find_all('a')

    try:
        total_item = int(page_item[4].text)
    except Exception:
        pass
        try:
            total_item = int(page_item[3].text)
        except Exception:
            pass
            try:
                total_item = int(page_item[2].text)
            except Exception:
                pass
                try:
                    total_item = int(page_item[1].text)
                except Exception:
                    pass
                    try:
                        total_item = int(page_item[0].text)
                    except Exception:
                        pass
    return 1 + total_item

# Game Scraping

In [3]:
def scrap_game_detail(game):# game_rating_descriptors, game_rating_icons, achievement_block, game_area_metascore
    count = 0
    datas = []

    for j in range(1, get_pagination(game)):
        param = {
            'term': game,
            'page': j,
        }
        req = requests.get(link, params=param, headers=head)
        soup = BeautifulSoup(req.text, 'html.parser')

        conten = soup.find('div', {'id': 'search_resultsRows'}).find_all('a')#tablodaki her bir oyun için
        for i in conten:
            #search page
            #url
            url = i['href']
            
            #title
            title = i.find('div', 'col search_name ellipsis').text.strip().replace('\n', ' ')

            #price
            try:
                price = i.find('div', 'col search_price responsive_secondrow').text.strip()
            except Exception:
                price = 'discount from ' + i.find('span', {'style': 'color: #888888;'}).text.replace(' ', '.') + ' to ' + i.find('div', 'col search_price discounted responsive_secondrow').find('br').next_sibling.strip() + f" ({i.find('div', 'col search_discount responsive_secondrow').text.replace('-', '').strip()})"
            if price == '':
                price = 'none'

            #release
            release = i.find('div', 'col search_released responsive_secondrow').text
            if release == '':
                release = 'none'

            #game page
            req_piece = requests.get(url, headers=head)
            soup_piece = BeautifulSoup(req_piece.text, 'html.parser')

            #description
            description = soup_piece.find("div", {"class" : "game_description_snippet"})
            if description == None:
                description = None 
            else:
                description = description.text.strip().replace('\n', ' ')

            #header image
            header_image = soup_piece.find("img", {"class" : "game_header_image_full"})
            if header_image == None:
                header_image = None
            else:
                header_image = header_image['src']
            
            #developer
            try:
                developer = soup_piece.find_all("div", {"class" : "dev_row"})[0]
                if developer == None:
                    developer = None
                else:
                    developer = developer.find("a").text
            except:
                developer = None
            
            #publisher
            try:
                publisher = soup_piece.find_all("div", {"class" : "dev_row"})[1]
                if developer == None:
                    publisher = None
                else:
                    publisher = publisher.find("a").text
            except:
                publisher = None
            
            #tags
            tags = soup_piece.find("div", {"class" : "glance_tags popular_tags"})
            if tags == None:
                tags = None
            else:
                tags = [i.text.strip().replace('\n', ' ') for i in tags.find_all("a")][:5]
              
            #languages
            try:  
                languages = soup_piece.find_all("div", {"id" : "languageTable"})
                if languages == None:
                    languages = None
                else:                    
                    languages = [i.text.strip().replace('\n', ' ') for i in languages[0].find_all("td", {"class":"ellipsis"})]
            
            except:
                languages = None
            
            #background image
            background_image = soup_piece.find("img", {"class" : "game_page_background game"})
            if background_image == None:
                background_image = None
            else:
                background_image = cssutils.parseStyle(background_image["style"])['background-image']
                background_image = background_image.replace('url(', '').replace(')', '')
                
            sys_req = soup_piece.find("div", {"class" : "game_area_sys_req sysreq_content active"})
            if sys_req == None:
                sys_req = None
            else:
                sys_req = sys_req.text.strip().replace('\n', ' ')
                
            #PEGI Rating
            rating_descriptors = soup_piece.find("div", {"class" : "game_rating_descriptors"}).text.split(",")
            rating_descriptors = [item.strip() for item in rating_descriptors]
            
            rating_image = soup_piece.find("div", {"class" : "game_rating_icon"}).find("img")["src"]
            
            #Achievement Count
            achievement_block = soup_piece.find("div", {"id":"achievement_block"}).find("div", {"class":"block_title"}).text.strip()
            
            #Metacritic Score
            metacritic_score = soup_piece.find("div", {"id":"game_area_metascore"}).find("div", {"class":"score high"}).text.strip()
            
            #DLC
            all_dlc_list = soup_piece.find("div", {"class":"game_area_dlc_list"}).find_all("a", {"class":"game_area_dlc_row odd ds_collapse_flag ds_collapse_flag_tiny"})
            
            dlc_name_list = [item.find("div", {"class":"game_area_dlc_name"}).text.strip() for item in all_dlc_list]
            dlc_link_list = [item['href'] for item in all_dlc_list]
            dlc_price_list = [item.find("div", {"class":"game_area_dlc_price"}).text.strip() for item in all_dlc_list]
            
            dlc_info = {"dlc_name_list" : dlc_name_list, "dlc_link_list" : dlc_link_list, "dlc_price_list" : dlc_price_list}

            data = {
                'title': title,
                'description': description,
                'price': price,
                'release': release,
                'link': url,
                'header_image': header_image,
                'developer': developer,
                'publisher': publisher,
                'tags':tags,
                'languages':languages,
                'background_image': background_image,
                'sys_req':sys_req
            }
            datas.append(data)

            count += 1
    return datas

In [4]:
def game_loop(gameList):
    all_game = list()
    for game in gameList:
        game_detail = scrap_game_detail(game)
        all_game.append(game_detail)
        
    return all_game

In [169]:
df = pd.read_csv("imdb_video_game_rating.csv")

In [193]:
gameList = df['title'].tolist()[:5]

In [ ]:
game_loop(gameList)

In [5]:
scrap_game_detail(game)

[{'title': 'God of War',
  'description': 'His vengeance against the Gods of Olympus years behind him, Kratos now lives as a man in the realm of Norse Gods and monsters. It is in this harsh, unforgiving world that he must fight to survive… and teach his son to do the same.',
  'price': '329,00 TL',
  'release': '14 Jan, 2022',
  'link': 'https://store.steampowered.com/app/1593500/God_of_War/?snr=1_7_7_151_150_1',
  'header_image': 'https://cdn.akamai.steamstatic.com/steam/apps/1593500/header.jpg?t=1650554420',
  'developer': 'Santa Monica Studio',
  'publisher': 'PlayStation PC LLC',
  'tags': ['Action', 'Singleplayer', 'Adventure', 'Story Rich', 'Mythology'],
  'languages': ['English',
   'French',
   'Italian',
   'German',
   'Spanish - Spain',
   'Dutch',
   'Japanese',
   'Korean',
   'Polish',
   'Portuguese - Portugal',
   'Portuguese - Brazil',
   'Russian',
   'Spanish - Latin America',
   'Turkish',
   'Traditional Chinese',
   'Czech',
   'Hungarian',
   'Greek'],
  'backgro

In [70]:
url = "https://store.steampowered.com/app/292030/The_Witcher_3_Wild_Hunt/"

req_piece = requests.get(url, headers=head)
soup_piece = BeautifulSoup(req_piece.text, 'html.parser')
#description

#PEGI Rating
rating_descriptors = soup_piece.find("div", {"class" : "game_rating_descriptors"}).text.split(",")
rating_descriptors = [item.strip() for item in rating_descriptors]

rating_image = soup_piece.find("div", {"class" : "game_rating_icon"}).find("img")["src"]

#Achievement Count
achievement_block = soup_piece.find("div", {"id":"achievement_block"}).find("div", {"class":"block_title"}).text.strip()

#Metacritic Score
metacritic_score = soup_piece.find("div", {"id":"game_area_metascore"}).find("div", {"class":"score high"}).text.strip()

#DLC
all_dlc_list = soup_piece.find("div", {"class":"game_area_dlc_list"}).find_all("a", {"class":"game_area_dlc_row odd ds_collapse_flag ds_collapse_flag_tiny"})

dlc_name_list = [item.find("div", {"class":"game_area_dlc_name"}).text.strip() for item in all_dlc_list]
dlc_link_list = [item['href'] for item in all_dlc_list]
dlc_price_list = [item.find("div", {"class":"game_area_dlc_price"}).text.strip() for item in all_dlc_list]

dlc_info = {"dlc_name_list" : dlc_name_list, "dlc_link_list" : dlc_link_list, "dlc_price_list" : dlc_price_list}

print(dlc_info)


{'dlc_name_list': ['The Witcher 3: Wild Hunt - Expansion Pass', 'The Witcher 3: Wild Hunt - Temerian Armor Set', 'The Witcher 3: Wild Hunt - Alternative Look for Yennefer', 'The Witcher 3: Wild Hunt - Nilfgaardian Armor Set', "The Witcher 3: Wild Hunt - New Quest 'Fool's Gold'", "The Witcher 3: Wild Hunt - New Quest: 'Scavenger Hunt: Wolf School Gear'", "The Witcher 3: Wild Hunt - New Quest 'Contract: Skellige's Most Wanted'", 'The Witcher 3: Wild Hunt - Alternative Look for Ciri', 'The Witcher 3: Wild Hunt - New Finisher Animations', 'The Witcher 3: Wild Hunt - Blood and Wine', 'The Witcher 3: Wild Hunt - Hearts of Stone Soundtrack'], 'dlc_link_list': ['https://store.steampowered.com/app/355880/The_Witcher_3_Wild_Hunt__Expansion_Pass/?snr=1_5_9__405', 'https://store.steampowered.com/app/370000/The_Witcher_3_Wild_Hunt__Temerian_Armor_Set/?snr=1_5_9__405', 'https://store.steampowered.com/app/370002/The_Witcher_3_Wild_Hunt__Alternative_Look_for_Yennefer/?snr=1_5_9__405', 'https://store.s